# Keys and Addresses Exercises

### Requirements

    $ pip3 install ethereum
    
    $ pip3 install bitcoin
    
    $ pip3 install pycryptodomex
    
    $ pip3 install jupyter


In [13]:
# Import libraries

import sys

# Vitalik Buterin's Python Library for Bitcoin
# No longer maintained!
# https://pypi.python.org/pypi/bitcoin/1.1.42
import bitcoin

# Vitalik Buterin's Python Library for Ethereum
# https://github.com/ethereum/pyethereum
import ethereum

# pysha3 package - SHA-3 (Keccak) for Python 2.7 - 3.5
# The sha3 module monkey patches the hashlib module.
# The monkey patch is automatically activated with the first import of the sha3 module.
if sys.version_info < (3, 6):
    import sha3

# Wrong source of SHA3 (FIPS-202 not Keccak-256)
from hashlib import sha3_256 as hashlib_sha3

# Both FIP-202 SHA-3 and Keccak-256 from pycryptodomex
from Crypto.Hash import SHA3_256 as crypto_sha3
from Crypto.Hash import keccak as crypto_keccak

# Ethereum library implements Keccak, but calls it sha3
from ethereum.utils import sha3 as keccak256

from rlp.utils import decode_hex, encode_hex


In [14]:
privkey_hex = "f8f8a2f43c8376ccb0871305060d7b27b0554d2cc72bccf41b2705608452f315"

In [15]:
privkey = decode_hex(privkey_hex)

In [16]:
# Use pybitcointools (bitcoin) library's elliptic curve functions to calculate the public key

pubkey = bitcoin.privtopub(privkey)

In [17]:
pubkey_hex = encode_hex(pubkey)

In [18]:
print("Public Key: " + pubkey_hex)

Public Key: 046e145ccef1033dea239875dd00dfb4fee6e3348b84985c92f103444683bae07b83b5c38e5e2b0c8529d7fa3f64d46daa1ece2d9ac14cab9477d042c84c32ccd0


In [19]:
pubkey_without_prefix = pubkey_hex[2:]
x_hex = pubkey_without_prefix[:64]
y_hex = pubkey_without_prefix[64:]
print("x (hex) : " + x_hex)
print("y (hex) : " + y_hex)

x = int(x_hex, 16)
y = int(y_hex, 16)
print("x (int) : ", x)
print("y (int) : ", y)

x (hex) : 6e145ccef1033dea239875dd00dfb4fee6e3348b84985c92f103444683bae07b
y (hex) : 83b5c38e5e2b0c8529d7fa3f64d46daa1ece2d9ac14cab9477d042c84c32ccd0
x (int) :  49790390825249384486033144355916864607616083520101638681403973749255924539515
y (int) :  59574132161899900045862086493921015780032175291755807399284007721050341297360


In [20]:
# Prove pubkey is a point on the curve

# p is the prime order of the elliptic curve field
p = 115792089237316195423570985008687907853269984665640564039457584007908834671663

(x ** 3 + 7 - y**2) % p

0

In [21]:
# Which "SHA3" am I using?

# Uncomment below to try various options
#test_hash = hashlib_sha3(b"").hexdigest()
#test_hash = crypto_sha3.new(b"").hexdigest()
#test_hash = crypto_keccak.new(digest_bits=256, data=b"").hexdigest()
test_hash = encode_hex(keccak256(b""))


print(test_hash)

if test_hash == "c5d2460186f7233c927e7db2dcc703c0e500b653ca82273b7bfad8045d85a470":
    print("Hash Function is Keccak-256")
elif test_hash == "a7ffc6f8bf1ed76651c14756a061d662f580ff4de43b49fa82d80a4b80f8434a":
    print("Hash Function is FIP-202 SHA-3")
else: 
    print("Oops! Can't identify SHA3")



c5d2460186f7233c927e7db2dcc703c0e500b653ca82273b7bfad8045d85a470
Hash Function is Keccak-256


In [22]:
hex_hash = encode_hex(keccak256(decode_hex(pubkey_without_prefix)))
print ("Public Key Hash: " + hex_hash)

Public Key Hash: 2a5bc342ed616b5ba5732269001d3f1ef827552ae1114027bd3ecf1f086ba0f9


In [23]:
address = hex_hash[24:]
print("Ethereum Address: 0x" + address)

Ethereum Address: 0x001d3f1ef827552ae1114027bd3ecf1f086ba0f9


In [24]:
# Let's calculate the EIP-55 mixed-capitalization checksum address
# Take the lower-case address and hash it again, to produce a checksum

address_hash_hex = encode_hex(keccak256(address))
print(address_hash_hex)

23a69c1653e4ebbb619b0b2cb8a9bad49892a8b9695d9a19d8f673ca991deae1


In [25]:
# Simple implementation of EIP-55
# For every alphabetic character of the address, 
# capitalize it if the corresponding character of the hash is greater than 8, 

a = ""
for i, c in enumerate(address):
    if c in '0123456789':
        a = a + c 
    elif int(address_hash_hex[i], 16) >= 8:
        a = a + c.upper()
    else:
        a = a + c.lower()
        
print("EIP-55 encoded Ethereum Address: 0x"+a)

EIP-55 encoded Ethereum Address: 0x001d3F1ef827552Ae1114027BD3ECF1f086bA0F9
